In [23]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
import re
from itertools import repeat
from keras.models import Sequential
from keras.layers import Dense
import keras

In [24]:
df = pd.DataFrame()
df  = pd.read_csv('labels_imperium_all.csv',encoding='utf-8',header=None)
labels = df[0]
df.head()

,0
0,1
1,0
2,0
3,0
4,0


In [25]:
df = pd.DataFrame()
df  = pd.read_csv('imperium_txts_all.csv',encoding='utf-8',header=None)
df.head()

,0
0,"""You fuck your dad."""
1,"""i really don't understand your point.\xa0 It ..."
2,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,"""listen if you dont wanna get married to a man..."
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [26]:
documents = df[0]
print(documents[0])
# lower case
documents = documents.str.lower() 
# remove punctuation
documents = documents.str.replace('[^\w\s]','') 
documents = documents.values.tolist()
for i in range(len(documents )):
    # tokenize
    documents [i] = nltk.word_tokenize(documents [i])
    # remove blanck spaces
    documents [i] = [re.sub(r'\W',' ',word) for word in documents [i] ]
print("Ther are :",len(documents)," 0:", documents[0])

"You fuck your dad."
Ther are : 8829  0: ['you', 'fuck', 'your', 'dad']


In [27]:
# After our pre-processing, there are not mpty documents

In [28]:
wordfreq = {}
for sentence in documents:
    for token in sentence:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print("Ther are :",len(wordfreq)," unique words")

Ther are : 30675  unique words


In [29]:
# Use vectorize function of numpy 
length_checker = np.vectorize(len) 
  
# Find the length of each element 
documents_length = length_checker(documents) 
  
# Print the length of each element 
print(documents_length) 

[  4  15  64 ...   8  52 188]


In [30]:
# List of keys to be deleted from dictionary
selectedKeys = list() 
 
# Iterate over the dict and put to be deleted keys in the list
for (key, value) in wordfreq.items():
    if value < 5:
        selectedKeys.append(key)
    if len(key) < 3:
        selectedKeys.append(key)
    if len(key) > 30:
        selectedKeys.append(key)
 
# Iterate over the list and delete corresponding key from dictionary
for key in selectedKeys:
    if key in wordfreq:
        del wordfreq[key]
print("Ther are :",len(wordfreq)," unique words")

Ther are : 4110  unique words


In [31]:
# Saving each unique word index
word_idx = {}
idx = 0
for token in wordfreq:
    word_idx[token] =  idx
    idx += 1
print(word_idx['boy'])

504


In [32]:
sentence_vectors = []
for sentence in documents:
    sent_vec = []
    for token in wordfreq:
        if token in sentence:
            df = sentence.count(token)
            sent_vec.append(df)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [34]:
sentence_vectors = np.asarray(sentence_vectors)
print("Matrix of dimension: (",sentence_vectors.shape[0],",",sentence_vectors.shape[1],")")

Matrix of dimension: ( 8829 , 4110 )


In [35]:
i = 0
a = 0
for row in sentence_vectors:
    if np.count_nonzero(row) == 0:
       # print(documents[i])
        a += 1
    i += 1
print(a)
# There are 71 documetns emptys

71


In [36]:
# Identifying the document with more terms, docs include terns not present in sentence_vectors
maxlen_doc = []
for doc in documents:
    maxlen_doc.append(len(doc))
print("Maximum document longitu (may include not present terms): ",max(maxlen_doc))  
print(max(documents_length))
maxlen_doc = (max(documents_length))

Maximum document longitu (may include not present terms):  2299
2299


In [37]:
# We need to separete this in sTrain and Stest, we need to be careful to mantein the index in the training fase
num_docs = sentence_vectors.shape[0]
#maximum = []
idx_matriz = np.zeros((num_docs, maxlen_doc)) # idx_matriz represents "stest" in Matlab
row = 0
for doc in documents:
    # If tern not in dicc, add '0'
    doc_inx = [ word_idx[x] if x in word_idx else 0 for x in doc] 
    doc_inx.extend(repeat(0, maxlen_doc - len(doc_inx)))  # Here we are padding, but it is NOT necessary
    idx_matriz[row] = (doc_inx)
    row += 1
    # Like this ingnore those words tht are not in dic : doc_inx = [wordfreq[x] for x in doc if x in wordfreq]
    #maximum.append(len(doc_inx))
print("Matrix of dimension: (",idx_matriz.shape[0],",",idx_matriz.shape[1],")")
# This is a matrix where each doc word is representa it by their indx , 0 is word not in dicc

Matrix of dimension: ( 8829 , 2299 )


In [38]:
#X_train = sentence_vectors[:6182]
#X_test  = sentence_vectors[6182:]
#Y_train = labels[:6182]
#Y_test  = labels[6182:]
#X_train, X_test, Y_train, Y_test = train_test_split(sentence_vectors, labels, test_size=0.3, random_state=42)
kf = KFold(n_splits=4)
kf.get_n_splits(sentence_vectors)
for train_index, test_index in kf.split(sentence_vectors):
    X_train, X_test = sentence_vectors[train_index], sentence_vectors[test_index]
    Y_train, Y_test = labels[train_index], labels[test_index]
    S_train, S_test = idx_matriz[train_index], idx_matriz[test_index]
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

(6622, 4110)
(6622,)
(2207, 4110)
(2207,)
(6622, 2299)
(2207, 2299)


PBR

In [39]:
def csa (X,Y,XT):
    num_docs = X.shape[0] # number of trainning docs
    num_terms = X.shape[1]
    num_class = 2
    TR = np.zeros((num_terms, num_class))
    DL = np.sum(X, axis=1) # sum each elemet row, wich is each df element of a document
    DL[DL == 0] = 1 #Those documents with not terms in dicc, are empty .
   
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X[docs,i] # tf_v is tern frecuency of each term
            TR[i][j] = sum(np.log2(1 + tf_v / DL [docs]))            
    print("PRB: (",TR.shape[0],",",TR.shape[1],")")

    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    n1[n1 == 0] = 1
    Tik = TR / n1

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        suma = sum(Tik[i])
        if suma == 0:
            suma = 1
        Tik[i] = Tik[i] / suma # Tik[i] = Tik[i] / sum(TR[i])

    print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X[i] != 0)[0] # extrac idxs =! 0
        a = X[i][termos] / DL[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos])) # alpha * tik; alpha is the df of each term
    # Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"    return Tik
    
    # Document Test Representation
    DLT = np.sum(XT, axis=1)
    DLT[DLT == 0] = 1
    num_docs = XT.shape[0]
    DRT = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_test[i] != 0)[0] 
        a = XT[i][termos] / DLT[i]
        a = np.expand_dims(a, axis=1)
        DRT[i] = sum(np.multiply(a,Tik[termos]))

    return DR,DRT

In [40]:
DR,DRT = csa (X_train,Y_train,X_test)
print("DR: (",DR.shape[0],",",DR.shape[1],")")

PRB: ( 4110 , 2 )
Tik: ( 4110 , 2 )
DR: ( 6622 , 2 )


In [41]:
#for i in DRT:
#    print(i)

In [42]:
# Create NN model
num_class = 2
model = Sequential()
# Add model layers
model.add(Dense(10, activation='relu', input_shape=(num_class,)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [44]:
model.fit(DR, Y_train, epochs=100, batch_size=10, validation_data=(DRT, Y_test))

Train on 6622 samples, validate on 2207 samples
Epoch 1/100
6622/6622 [==============================] - 2s 337us/step - loss: 0.6589 - acc: 0.6201 - val_loss: 0.5915 - val_acc: 0.7349
Epoch 2/100
6622/6622 [==============================] - 1s 158us/step - loss: 0.6275 - acc: 0.6626 - val_loss: 0.5810 - val_acc: 0.7349
Epoch 3/100
6622/6622 [==============================] - 1s 153us/step - loss: 0.6159 - acc: 0.6626 - val_loss: 0.5701 - val_acc: 0.7349
Epoch 4/100
6622/6622 [==============================] - 1s 159us/step - loss: 0.5980 - acc: 0.6632 - val_loss: 0.5571 - val_acc: 0.7367
Epoch 5/100
6622/6622 [==============================] - 1s 168us/step - loss: 0.5721 - acc: 0.6753 - val_loss: 0.5396 - val_acc: 0.7599
Epoch 6/100
6622/6622 [==============================] - 1s 166us/step - loss: 0.5379 - acc: 0.7194 - val_loss: 0.5103 - val_acc: 0.7685
Epoch 7/100
6622/6622 [==============================] - 1s 155us/step - loss: 0.4988 - acc: 0.7703 - val_loss: 0.4766 - val_acc: 

6622/6622 [==============================] - 1s 153us/step - loss: 0.3358 - acc: 0.8487 - val_loss: 0.4244 - val_acc: 0.8210
Epoch 61/100
6622/6622 [==============================] - 1s 176us/step - loss: 0.3356 - acc: 0.8493 - val_loss: 0.4239 - val_acc: 0.8206
Epoch 62/100
6622/6622 [==============================] - 1s 181us/step - loss: 0.3356 - acc: 0.8499 - val_loss: 0.4227 - val_acc: 0.8183
Epoch 63/100
6622/6622 [==============================] - 1s 160us/step - loss: 0.3358 - acc: 0.8478 - val_loss: 0.4229 - val_acc: 0.8197
Epoch 64/100
6622/6622 [==============================] - 1s 162us/step - loss: 0.3354 - acc: 0.8481 - val_loss: 0.4233 - val_acc: 0.8188
Epoch 65/100
6622/6622 [==============================] - 1s 158us/step - loss: 0.3349 - acc: 0.8506 - val_loss: 0.4236 - val_acc: 0.8228
Epoch 66/100
6622/6622 [==============================] - 1s 159us/step - loss: 0.3357 - acc: 0.8487 - val_loss: 0.4232 - val_acc: 0.8224
Epoch 67/100
6622/6622 [=======================

In [51]:
#retains = [0.0100000000000000,0.0600000000000000,0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]
#retains = [0.0100000000000000]
retains = [0,0.05,0.1,0.15,0.20,0.2,0.30,0.35,0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,1]
for j in range (0,len(retains)):
    print("---------------------------------------- ",retains[j]," ---------------------------------------------")
    rXtest = np.zeros((X_test.shape[0], X_test.shape[1]))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(X_test.shape[1])  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                freqtsof[int(S_test[i,wdix])] += 1 #If the same word appears more than once , just add one to the position
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
        
    DRS,DR_TS = csa(X_train,Y_train,rXtest)
    #DRS = np.nan_to_num(DRS)
    #DR_TS = np.nan_to_num(DR_TS)
    print(DRS.shape)
    print(DR_TS.shape)
    his = model.fit(DRS, Y_train, epochs=30, batch_size=10, validation_data=(DR_TS, Y_test))
    #Xtestv = rXtest

----------------------------------------  0  ---------------------------------------------
PRB: ( 4110 , 2 )
Tik: ( 4110 , 2 )
(6622, 2)
(2207, 2)
Train on 6622 samples, validate on 2207 samples
Epoch 1/30
6622/6622 [==============================] - 1s 162us/step - loss: 0.3249 - acc: 0.8487 - val_loss: 0.5904 - val_acc: 0.7349
Epoch 2/30
6622/6622 [==============================] - 1s 154us/step - loss: 0.3248 - acc: 0.8503 - val_loss: 0.6017 - val_acc: 0.7349
Epoch 3/30
6622/6622 [==============================] - 1s 161us/step - loss: 0.3248 - acc: 0.8469 - val_loss: 0.5824 - val_acc: 0.7349
Epoch 4/30
6622/6622 [==============================] - 1s 182us/step - loss: 0.3243 - acc: 0.8487 - val_loss: 0.5972 - val_acc: 0.7349
Epoch 5/30
6622/6622 [==============================] - 1s 170us/step - loss: 0.3250 - acc: 0.8476 - val_loss: 0.5846 - val_acc: 0.7349
Epoch 6/30
6622/6622 [==============================] - 1s 151us/step - loss: 0.3245 - acc: 0.8487 - val_loss: 0.5844 - val_a

KeyboardInterrupt: 

In [ ]:
X_test.shape